In [7]:
import redis
import hiredis
import ranking
import os
import evaluate
import conn
from rq import Queue
import ast
from tournament import Team, Game
import pprint
import random
import statistics as stats
import math
import mock
pp = pprint.PrettyPrinter(indent=4)
pprint = pp.pprint
redis_host = 'localhost'
redis_port = '6379'
wait_time = '600'
r = redis.StrictRedis(host=redis_host, port=redis_port, db=0)
session = {'s_ind':1}
s_id = session['s_ind']
session_key = "".join(["s:",str(s_id)])

dc = conn.SessionConnection(session_key,r)

In [59]:
reload(conn)

<module 'conn' from 'conn.py'>

In [9]:
print(dc.games_list_key())
print(dc.team_list_key())
print(dc.game_key(1))
print(dc.strength_key(1))
print(dc.property_key(1,'team_1'))
dc.r_conn.lrange(dc.team_list_key(),0,-1)

s:1:game_ids
s:1:t_ids:
s:1:game:1
s:1:strengths:1
s:1:game:1:team_1


[]

In [3]:
def load_teams():
    file = teams_file
    pass
def initilize_database():
    wc_day_1am = {'id':1,'name':'Wildcard: Day 1 AM','type':'swiss'}
    wc_day_1pm = {'id':2,'name':'Wildcard: Day 1 PM','type':'swiss'}
    wc_day_2am = {'id':3,'name':'Wildcard: Day 2 AM','type':'swiss'}
    wc_day_2pm = {'id':4,'name':'Wildcard: Day 2 PM','type':'swiss'}
    f_day_1am = {'id':5, 'name':'Worlds: Day 1 AM','type':'swiss'}
    f_day_1pm = {'id':6, 'name':'Worlds: Day 1 PM','type':'swiss'}
    f_day_2am = {'id':7, 'name':'Worlds: Day 2 AM','type':'swiss'}
    f_day_2pm = {'id':8, 'name':'Worlds: Day 2 PM','type':'swiss'}
    f_day_3groups = {'id':9, 'name':'Worlds: Day 3 Group Stage','type':'group'}
    f_day_3knockout = {'id':10, 'name':'Worlds: Day 3 Knockout Stage','type':'knockout'} 

    sessions = [wc_day_1am, wc_day_1pm,wc_day_2am,wc_day_2pm,
                day_1am,day_1pm,day_2am,day_2pm,day_3groups,day_3knockout]
    
    
    

In [10]:
def initilize_session(s_id,name,team_ids,r_conn)
    session_key = "".join(["s:",str(s_id)])
    session_name_key = "".join([session_key,'name'])
    r.set(session_name_key,name)
    sc = conn.SessionConnection(session_key,r_conn)
    r_conn.set(sc.new_game_id_key(),0)
    
    for id in team_ids:
        r.rpush(sc.team_list_key(),id)

    


-1

In [176]:
teams = list(set([g.teams[0].id for g in sess.games] + [g.teams[1].id for g in sess.games]))
d = {'a':1,'b':2,'c':3}
d.keys()

['a', 'c', 'b']

In [26]:
r.delete(dc.team_list_key())
r.delete(dc.games_list_key())
test_games = {0:[1,0,1,3,1,0,0,1],
              1:[3,2,1,3,1,0,0,1],
              2:[2,0,1,3,2,0,0,1],
              3:[3,1,1,5,2,0,0,1],
              4:[2,1,0,0,3,0,0,0]}
def make_game(id,props):
    game = dict(zip(['team_1','team_2','score_1','score_2',
                     'round','s_updated','kl_updated','status'],props))
    game['id'] = id
    return game

for k in test_games:
   dc.set_game(make_game(k,test_games[k])) 
   r.rpush(dc.games_list_key(),k)

def set_team_ids(ids):
    for id in ids:
        r.rpush(dc.team_list_key(),id)
    
set_team_ids([0,1,2,3,4])
teams = dc.make_teams(dc.get_team_list())
games = dc.get_games()
sess = ranking.Seeding(teams)
sess.games = dc.mcmc_games(games,teams)
sess.games[4].scores
sess.mc_points=1000
sess.mc_burn = 100
sess.fit_model()

running MCMC
 [-----------------100%-----------------] 1000 of 1000 complete in 1.2 sec

In [29]:
reload(evaluate)
reload(ranking)
reload(conn)
evaluate.update_results(session_key,r)
evaluate.update_kl_info(session_key,r)
evaluate.get_best_staging(session_key,r,3)


running MCMC
 [-----------------100%-----------------] 1000 of 1000 complete in 1.0 secrunning MCMC
 [-----------------100%-----------------] 1000 of 1000 complete in 0.8 sec

0

In [205]:
dc.get_kl_vec(range(4*3/2))
teams = dc.make_teams(dc.get_team_list())
sess=ranking.Seeding(teams)
sess = evaluate.init_session(teams)
dc.get_kl_vec(range(4*3/2))


[<tournament.Team instance at 0x10b3500e0>,
 <tournament.Team instance at 0x10b3506c8>,
 <tournament.Team instance at 0x10b3505a8>,
 <tournament.Team instance at 0x10b350710>,
 <tournament.Team instance at 0x10b350758>]

In [17]:
s = evaluate.fit_session(dc)
s.strengths()

running MCMC
 [-----------------100%-----------------] 1000 of 1000 complete in 1.1 sec

{0: 0,
 1: -0.0019102709897806755,
 2: 0.0015336655197930539,
 3: 0.0036279090503453324,
 4: 5.3916536667117094e-05}

In [21]:
strengths = dict(zip([t.id for t in s.teams],s.strengths()))
strengths.

{0: 0, 1: 1, 2: 2, 3: 3, 4: 4}

In [13]:
evaluate.update_kl_info(session_key,r)


running MCMC
 [-----------------100%-----------------] 1000 of 1000 complete in 1.1 sec

0

In [77]:
new_game_id_key = "".join([session_key,"new_game_id"])
def new_game_id():
    id = r.get(new_game_id_key)
    r.incr(new_game_id_key)
    return int(id)
r.set(new_game_id_key,0)


True

In [130]:
sess=evaluate.fit_session(dc)
sess.strengths()
strengths = dict(zip([t.id for t in sess.teams],sess.strengths()))
for id in strengths:
    print(id)
    print(dc.strength_key(id))

running MCMC
 [-----------------100%-----------------] 1000 of 1000 complete in 0.6 sec0
s:1:strengths:0
1
s:1:strengths:1
2
s:1:strengths:2
3
s:1:strengths:3
4
s:1:strengths:4


In [26]:
reload(mock)
reload(ranking)
s = mock.generate_session(8)
s.mc_points = 10000
s.mc_burn = 100
s = mock.generate_round(s,1)
s.fit_model()
s = mock.generate_round(s,2)
s.fit_model()
s = mock.generate_round(s,3)
s.fit_model()

running MCMC
 [-----------------100%-----------------] 10000 of 10000 complete in 5.8 secrunning MCMC
 [-----------------100%-----------------] 10000 of 10000 complete in 6.0 secrunning MCMC
 [-----------------100%-----------------] 10000 of 10000 complete in 6.3 sec

In [28]:
s = generate_session(28)
s.mc_points = 10000
s.mc_burn = 1000
s = generate_round(s,1)
s.fit_model()

%time games = s.update_KL_graph()


running MCMC
 [-----------------100%-----------------] 10000 of 10000 complete in 3.4 secCPU times: user 45.2 s, sys: 83.8 ms, total: 45.3 s
Wall time: 45.2 s


In [29]:
s = generate_session(28)
s.mc_points = 100000
s.mc_burn = 10000
s = generate_round(s,1)
s.fit_model()

%time games = s.update_KL_graph()


running MCMC
 [-----------------100%-----------------] 100000 of 100000 complete in 34.4 secCPU times: user 7min 3s, sys: 182 ms, total: 7min 3s
Wall time: 7min 3s


In [149]:
id = dc.new_game_id()
id

In [10]:
r.flushall()
r.set(dc.new_game_id_key(),0)

True

In [64]:
f = kl_func(s)
%time parmap(f,range(378))
%time s.fit_model()

CPU times: user 933 ms, sys: 121 ms, total: 1.05 s
Wall time: 5min 21s
running MCMC
 [-----------------100%-----------------] 100000 of 100000 complete in 35.8 secCPU times: user 40.4 s, sys: 27.3 s, total: 1min 7s
Wall time: 35.8 s


In [29]:
pprint([[t.id, t.strength, s.strengths()[ind]] for ind,t in enumerate(s.teams)])
math.sqrt(sum([([t.strength for t in s.teams][ind]-st)**2 for ind,st in enumerate(s.strengths())]))/s.n_teams

[   [0, 0.3752943620190973, 0.45426442959151403],
    [1, -1.6376178714075802, -2.3115184423612858],
    [2, -0.11921177384034226, 0.81934107408149992],
    [3, 0.44828581135608675, 0.054955749685839814],
    [4, 0.6310517958122059, 0.26132250485710684],
    [5, 1.196535761598349, 1.8662282575723685],
    [6, 0.6900951751681348, -0.012526869477989833],
    [7, -1.584433260705951, -1.1320667039490466]]


0.20839675440988437

In [63]:
len(s.marginal_ind)

378

In [24]:
[[[t.id for t in g.teams],g.scores,g.round] for g in s.games]

[[[0, 7], [5, 1], 1],
 [[1, 6], [0, 0], 1],
 [[2, 5], [0, 0], 1],
 [[3, 4], [2, 9], 1]]

In [860]:
pr_f = lambda x: math.exp(s.prob_func(s.teams[3].strength-s.teams[1].strength,1.0,x[0],x[1]))

In [861]:
[discrete_lahiri(pr_f,score_choices(10)) for x in range(20)]

[[5, 2],
 [1, 0],
 [3, 2],
 [3, 4],
 [0, 6],
 [3, 5],
 [0, 6],
 [9, 6],
 [2, 5],
 [9, 9],
 [0, 0],
 [4, 0],
 [8, 6],
 [0, 6],
 [2, 3],
 [6, 6],
 [6, 0],
 [0, 3],
 [8, 9],
 [5, 5]]

In [859]:
s = generate_session(4)
s.scale

2.0

In [391]:
[t.id for t in sess.stage_round(3)[1].teams]

[2, 4]

In [892]:
x = random.choice(score_choices(10))
p = pr_f(x)
u = random.uniform(0,1)
print(x)
print(p)
print(u)

[0, 2]
0.0561291416822
0.787541383464


In [900]:
reload(ranking)
reload(worker)
reload(evaluate)
reload(conn)

<module 'conn' from 'conn.pyc'>

In [918]:
import rq
import worker
import evaluate
import time
import conn
import statistics as stats

In [950]:
queue = rq.Queue(connection=redis.Redis())
s.mc_points=100000
s.mc_burn = 10000
s.fit_model()


running MCMC
 [-----------------100%-----------------] 100000 of 100000 complete in 80.0 sec

In [520]:
job = queue.enqueue_call(func=evaluate.update_results,args=(sess.games,sess))

In [519]:
job = queue.enqueue(evaluate.test_func)

In [642]:
evaluate.update_results(session_key,r)

running MCMC
 [-----------------100%-----------------] 1000 of 1000 complete in 0.6 sec

AttributeError: 'int' object has no attribute 'info'

In [740]:
filter(lambda x: not x == 5,range(10))

[0, 1, 2, 3, 4, 6, 7, 8, 9]

In [468]:
print(job.result)

None


In [489]:
queue.get_job_ids()

[u'848e283d-0d6b-47e4-a7a1-9b37899dba0c']

In [524]:
import pickle

In [38]:
r.hset('myhash','t1',1)
r.hset('myhash','t2',2)
r.hset('myhash','val',2.12321)
d = r.hgetall('myhash')
for key,val in d.iteritems():
    d[key] = num(val)
d


{'t1': 1, 't2': 2, 'val': 2.12321}

In [36]:
d={}

0.6029027210959689

In [183]:
a = [1,2,3,4,'a','b','c']
a.index('d')

ValueError: 'd' is not in list

<function numpy.core.fromnumeric.mean>

In [176]:
import csv

Data = {}

wct = []
with open('/Users/achaywa/Developer/clojure/re-frame/swiss/wildcard_teams.csv', 'rb') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',')
    for row in spamreader:
        wct.append(filter(lambda x:x is not '',row))
wc_teams = []
for ind,line in enumerate(wct[1:]):
    wc_teams.append({'name':line[1],'id':ind,'session':line[2],'wc':True})

ft = []
with open('/Users/achaywa/Developer/clojure/re-frame/swiss/finals_teams.csv', 'rb') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',',quotechar='"')
    for row in spamreader:
        ft.append(filter(lambda x:x is not '',row))

f_teams = []
for ind,line in enumerate(ft):
    if not line[0] == '0':
        f_teams.append({'name':line[0],'id':ind+28,'session':'A','wc':False})
for ind,line in enumerate(ft):
        if not line[1] == '0':
            f_teams.append({'name':line[1],'id':ind+13+28,'session':'B','wc':False})

        teams = wc_teams + f_teams
team_dict = dict(zip([t['id'] for t in teams],teams))


with open("".join([pwd,'/teams.csv']), 'wb') as csvfile:

    writer = csv.writer(csvfile,delimiter=',')
    for line in team_dict.values():
           writer.writerow(line.values())


In [410]:
import csv
import os
DB = {}
pwd = os.getcwd()
team_file = "".join([pwd,'/teams.csv'])
def teams_reader():
    def kv(row):
        return int(row[3]), {'session':row[0],'wc':bool(row[1]=='True'),
                        'name':row[2],'id':int(row[3])}

    team_dict = {}
    with open(team_file, 'rb') as csvfile:
        reader = csv.reader(csvfile, delimiter=',')
        for row in reader:
            k,v = kv(row)
            team_dict[k] = v
    return team_dict


DB['Teams'] = teams_reader()

for v in DB['Teams'].values():
        v['strength'] = 0


def id_to_name(id):
    return DB['Teams'][id]['name']

team_dict = DB['Teams']



wc_teams = [k for k,v in team_dict.iteritems() if v['wc']]
f_teams = [k for k,v in team_dict.iteritems() if (not v['wc'])]
wc_teamsA = [k for k,v in team_dict.iteritems() if v['wc'] and (v['session']=='A')]
wc_teamsB = [k for k,v in team_dict.iteritems() if v['wc'] and (v['session']=='B')]
f_teamsA = [k for k,v in team_dict.iteritems() if (not v['wc']) and (v['session']=='A')]
f_teamsB = [k for k,v in team_dict.iteritems() if (not v['wc']) and (v['session']=='B')]

def initilize_sessions():
    wc_day_1am = {'id':1,'name':'Wildcard: Day 1 AM','type':'swiss','teams':wc_teamsA}
    wc_day_1pm = {'id':2,'name':'Wildcard: Day 1 PM','type':'swiss','teams':wc_teamsB}
    wc_day_2am = {'id':3,'name':'Wildcard: Day 2 AM','type':'swiss','teams':[]}
    wc_day_2pm = {'id':4,'name':'Wildcard: Day 2 PM','type':'swiss','teams':[]}
    f_day_1am = {'id':5, 'name':'Worlds: Day 1 AM','type':'swiss','teams':f_teamsA}
    f_day_1pm = {'id':6, 'name':'Worlds: Day 1 PM','type':'swiss','teams':f_teamsB}
    f_day_2am = {'id':7, 'name':'Worlds: Day 2 AM','type':'swiss','teams':[]}
    f_day_2pm = {'id':8, 'name':'Worlds: Day 2 PM','type':'swiss','teams':[]}
    f_day_3groups = {'id':9, 'name':'Worlds: Day 3 Group Stage','type':'group','teams':[]}
    f_day_3knockout = {'id':10, 'name':'Worlds: Day 3 Knockout Stage','type':'knockout','teams':[]} 
    
    sessions = {1:wc_day_1am, 2:wc_day_1pm,3:wc_day_2am,4:wc_day_2pm,
                5:f_day_1am,6:f_day_1pm,7:f_day_2am,8:f_day_2pm,
                9:f_day_3groups,10:f_day_3knockout}
    for v in sessions.values():
        v['games'] = []
        v['suggestedGames'] = []
        v['strengths'] = {}
        for id in v['teams']:
            v['strengths'][id] = 0

    return sessions


DB['Sessions'] = initilize_sessions()

def initilize_games():
    games = {}
    return games

DB['Games'] = initilize_games()

courts = {ind:{'id':ind,
               'just_played':None,
               'on_court':None,
               'up_next':[],
               'name':''.join(['Court ',str(ind)])} for ind in range(1,4)}
DB['Courts'] = courts 

In [327]:
def print_team(d):
    pprint(''.join([str(d['id']),': ',d['name']]))

def print_game(d):
    if d['score_1'] and d['score_2']:
        pprint("".join([str(d['id']),':', 'r', str(d['round']),'  ',
                        str(d['score_1']),' -- ',
                        DB['Teams'][d['team_1']]['name'],
                        ' vs ', DB['Teams'][d['team_2']]['name'], ' -- ',
                        str(d['score_2'])]))
    else:
        pprint("".join([str(d['id']),':','r',str(d['round']),'  ',
                DB['Teams'][d['team_1']]['name'],
                ' vs ', DB['Teams'][d['team_2']]['name']]))
def list_teams(session):
    for id in session['teams']:
        print_team(DB['teams'][id])
def list_games(session):
    for id in session['games']:
        print_game(DB['Games'][id])

In [370]:
DB['Games'][0] = {'status':1,'id':1,'team_1':0,'team_2':1,'scores':{0:1,1:3},'score_2':2, 'round':1}
DB['Sessions'][1]['games'] = [0]
list_games(Sessions[1])

KeyError: 'score_1'

In [411]:
pwd

'/Users/achaywa/Developer/clojure/re-frame/swiss/python'

In [328]:
DB['Games'][1] = {'status':1,'id':1,'team_1':0,'team_2':1,'score_1':1,'score_2':2, 'round':1}
DB['Games'][2] = {'status':1,'id':2,'team_1':3,'team_2':2,'score_1':5,'score_2':2, 'round':1}
DB['Games'][3] = {'status':1,'id':3,'team_1':7,'team_2':0,'score_1':5,'score_2':2, 'round':1}

DB['Sessions'][1]['games'] = [1,2,3]
list_games(DB['Sessions'][1])

'1:r1  1 -- Ball Bangers vs Wabbit -- 2'
'2:r1  5 -- The Edinburgh Badgers vs Golem -- 2'
'3:r1  5 -- Sons of Odin vs Ball Bangers -- 2'


In [308]:
session = DB['Sessions'][1]


{'games': [1, 2, 3],
 'id': 1,
 'name': 'Wildcard: Day 1 AM',
 'teams': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13],
 'type': 'swiss'}

In [307]:
td = {1:{'a':1,'b':2}, 2:{'c':3,'d':4}}
x = td
x[1]=5
td

{1: 5, 2: {'c': 3, 'd': 4}}

In [265]:
import(bayes)
bayes.update_results(Sessions[1],Games)


running MCMC
 [-----------------100%-----------------] 1000 of 1000 complete in 1.4 secrunning MCMC
 [-----------------100%-----------------] 1000 of 1000 complete in 1.1 sec

{0: -0.0039697123261410753,
 1: -0.0016918360929196546,
 2: 0.0029168005153548736,
 3: -0.0020726793124049786,
 4: 0,
 5: 0,
 6: 0,
 7: 0.0048174272161108341,
 8: 0,
 9: 0,
 10: 0,
 11: 0,
 12: 0,
 13: 0}

In [269]:
[g.id for g in sess.stage_round(1,kl)]

[None, None, None, None, None, None, None]

In [293]:
reload(bayes)
kl = bayes.update_kl_info(Sessions[1],Games)
Data['suggested_games'] = bayes.get_best_staging(Sessions[1],Games,kl,2)

running MCMC
 [-----------------100%-----------------] 1000 of 1000 complete in 1.5 sec

In [299]:
[ print_game(g) for g in games.values()]

'0:r2  Ball Bangers vs Pete un Plomb'
'4:r2  Wabbit vs With Fire'
'5:r2  Golem vs Triple Fisted'
'6:r2  The Edinburgh Badgers vs The Budgie Smugglers'
'7:r2  JKL vs Daft Monks'
'8:r2  The Inseminators vs Rusty Spoons'
'9:r2  Gentlemans vs Sons of Odin'


[None, None, None, None, None, None, None]

In [347]:
import bayes

def add_game(db,session_id,game):
    #check id
    
    g_ids = [g['id'] for g in db['Games'].values()]
    if game['id'] in g_ids:
        game['id'] = max(g_ids) + 1

    db['Sessions'][session_id]['games'].append(game['id'])
    db['Games'][game['id']] = game
    return db

def update_strengths(db,session_id):
    sess = db['Sessions'][id]
    str_dict = bayes.update_results(Sessions[1],Games)
    for k,v in str_dict:
        sess['strengths'][k] = v
    return db

def update_score(db,game_id,score_dict):
    db['Games'][game_id]['scores']=score_dict
    db['Games'][game_id]['status']=1
    
    return db

def new_game(team_1,team_2,round):
    return {'team_1':team_1,'team_2':team_2,
            'round':round,'scores':{team_1:0,team_2:0},
            'status':0}

def get_suggested_games(db,session_id,round):
    kl = bayes.update_kl_info(Sessions[1],Games)
    games = bayes.get_best_staging(Sessions[1],Games,kl,2)
    db['Sessions'][session_id]['suggestedGames'] = games
    return db

def add_suggested_games(db,session_id):
    for g in db['Sessions'][session_id]['suggestedGames'].values():
        db = add_game(db,session_id,g)
    return db


running MCMC
 [-----------------100%-----------------] 1000 of 1000 complete in 0.6 sec

{0: {'id': 0,
  'round': 2,
  'score_1': None,
  'score_2': None,
  'status': 0,
  'team_1': 0,
  'team_2': 13},
 4: {'id': 4,
  'round': 2,
  'score_1': None,
  'score_2': None,
  'status': 0,
  'team_1': 1,
  'team_2': 12},
 5: {'id': 5,
  'round': 2,
  'score_1': None,
  'score_2': None,
  'status': 0,
  'team_1': 2,
  'team_2': 11},
 6: {'id': 6,
  'round': 2,
  'score_1': None,
  'score_2': None,
  'status': 0,
  'team_1': 3,
  'team_2': 10},
 7: {'id': 7,
  'round': 2,
  'score_1': None,
  'score_2': None,
  'status': 0,
  'team_1': 4,
  'team_2': 9},
 8: {'id': 8,
  'round': 2,
  'score_1': None,
  'score_2': None,
  'status': 0,
  'team_1': 5,
  'team_2': 8},
 9: {'id': 9,
  'round': 2,
  'score_1': None,
  'score_2': None,
  'status': 0,
  'team_1': 6,
  'team_2': 7}}

In [345]:
add_suggested_games(DB,1)

In [363]:
import req
import time
redis_host = 'localhost'#os.environ['REDIS_HOST']
redis_port = '6379'#os.environ['REDIS_PORT']
wait_time = 100#os.environ['WAIT_TIME']
r = redis.StrictRedis(host=redis_host, port=redis_port, db=0)
q = Queue(connection=redis.Redis())



In [394]:
reload(req)
reload(bayes)
job = q.enqueue_call(func=req.get_suggested_games,
                                args=(DB,1,1), timeout=wait_time)
job.result or 
time.sleep(10)  # be nice to the system :)
job.result

{'Courts': {1: {'id': 1,
   'just_played': None,
   'name': 'Court 1',
   'on_court': None,
   'up_next': []},
  2: {'id': 2,
   'just_played': None,
   'name': 'Court 2',
   'on_court': None,
   'up_next': []},
  3: {'id': 3,
   'just_played': None,
   'name': 'Court 3',
   'on_court': None,
   'up_next': []}},
 'Games': {-1: {'id': -1,
   'round': 1,
   'scores': {1: 0, 2: 5},
   'status': 1,
   'team_1': 1,
   'team_2': 2},
  0: {'id': 1,
   'round': 1,
   'score_2': 2,
   'scores': {0: 1, 1: 3},
   'status': 1,
   'team_1': 0,
   'team_2': 1},
  1: {'id': 1,
   'round': 1,
   'score_1': 1,
   'score_2': 2,
   'status': 1,
   'team_1': 0,
   'team_2': 1},
  2: {'id': 2,
   'round': 1,
   'score_1': 5,
   'score_2': 2,
   'status': 1,
   'team_1': 3,
   'team_2': 2},
  3: {'id': 3,
   'round': 1,
   'score_1': 5,
   'score_2': 2,
   'status': 1,
   'team_1': 7,
   'team_2': 0},
  4: {'id': 4,
   'round': 2,
   'score_1': None,
   'score_2': None,
   'status': 0,
   'team_1': 1,
   't

{'Courts': {1: {'id': 1,
   'just_played': None,
   'name': 'Court 1',
   'on_court': None,
   'up_next': []},
  2: {'id': 2,
   'just_played': None,
   'name': 'Court 2',
   'on_court': None,
   'up_next': []},
  3: {'id': 3,
   'just_played': None,
   'name': 'Court 3',
   'on_court': None,
   'up_next': []}},
 'Games': {-1: {'id': -1,
   'round': 1,
   'scores': {1: 0, 2: 5},
   'status': 1,
   'team_1': 1,
   'team_2': 2},
  0: {'id': 1,
   'round': 1,
   'score_2': 2,
   'scores': {0: 1, 1: 3},
   'status': 1,
   'team_1': 0,
   'team_2': 1},
  1: {'id': 1,
   'round': 1,
   'score_1': 1,
   'score_2': 2,
   'status': 1,
   'team_1': 0,
   'team_2': 1},
  2: {'id': 2,
   'round': 1,
   'score_1': 5,
   'score_2': 2,
   'status': 1,
   'team_1': 3,
   'team_2': 2},
  3: {'id': 3,
   'round': 1,
   'score_1': 5,
   'score_2': 2,
   'status': 1,
   'team_1': 7,
   'team_2': 0},
  4: {'id': 4,
   'round': 2,
   'score_1': None,
   'score_2': None,
   'status': 0,
   'team_1': 1,
   't

In [399]:
DB = lambda: None or DB
DB

<function __main__.<lambda>>

In [386]:
reload(req)
DB = req.add_game(DB,1,req.update_score(req.new_game(1,2,1),{1:0,2:5}))
DB = req.add_game(DB,1,req.update_score(req.new_game(3,4,1),{3:3,4:1}))
DB = req.add_game(DB,1,req.update_score(req.new_game(5,6,1),{5:2,6:5}))

In [406]:
class Test(object): # make sure you inherit from object
    
    
    def __init__(self):
        self.n=4
    @property   
    def x(self):
        return self.n
    
    def z(self,r):
        self.n = r
        
p = Test()
p.x # returns 4

4

In [407]:
y = p.x

In [409]:
p.z(5)
y

4